In [684]:
from config import sparqlTerms, mig_ns, sparql_mig_test, sparql_mig_simple, sparql_mig_dev, vocabs
from transformation import TransformationFactory
from SPARQLWrapper import JSON
from utilities import removeNS, PrintException
import concurrent.futures
import re

##  MAIN CONTROLLER

In [685]:
def main():
    for ptype in ["collection", "community", "generic", "thesis", "batch"]:
        mig = QueryFactory.getMigrationQuery(ptype)
        #for q in mig.queries:
        #    DTO = DataFactory.getData(q, mig)
        #    DTO.transformData()

##  TRANSFORMATIONS
#### functions for handling data passed over by the data object. Takes a triple, detects what kind of action needs to be taken based on the predicate, sends it to the appropriate function for transformations, then returns it back to the data handler to be saved.

In [686]:
class Transformation():
    
    """
    the output must be a list of triples matching the same format as the input (as follows):
    
    {
        'subject': {
            'value': 'http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/0r/96/76/28/0r967628d', 
            'type': 'uri'
        }, 
        'predicate': {
            'value': 'http://purl.org/dc/elements/1.1/subject', 
            'type': 'uri'
        }, 
        'object': {
            'value': 'Geochemistry', 
            'type': 'literal'
        }
    }
    output is appended to self.output
    """
    
    def __init__(self):
        self.output = []
        
    ############################################################################
    ######################## transformation on rdf:type ########################
    ############################################################################
        
    def rdfsyntaxnstype(self, triple, ptype):
        self.output.append(triple)
        return self.output

       
    ############################################################################
    ######################## transformation on dcterms:language ################
    ############################################################################

    def language(self, triple, ptype):
        # normalize values and convert to URI (consult the "vocabs" variable from the config file (this folder))
        self.output.append(triple)
        return self.output


    ############################################################################
    ######################## transformation on dc:rights #######################
    ############################################################################
    
    
    def rights(self, triple, ptype):
        #### 
        # several different license values need to be coerced into one common value, this needs to be confirmed with leah before it is written
        self.output.append(triple)
        return self.output

    
    ############################################################################
    ######################## transformation on ual:institution #################
    ############################################################################

    def institution(self, triple, ptype):
        # convert university of alberta to <http://id.loc.gov/authorities/names/n79058482>
        self.output.append(triple)
        return self.output
 

    ############################################################################
    ######################## transformation on dcterms:license #################
    ############################################################################

    
    def license(self, triple, ptype):
        #### 
        # convert licenses from text to URI (use vocabs variable, some coersion will be necessary)
        self.output.append(triple)
        return self.output
    
    
    ############################################################################
    ######################## transformation on dcterms:type ####################
    ############################################################################
    
    def type(self, triple, ptype):      
        if ptype == 'batch':
                # null
                # Complete
                # processing
            self.output.append(
                 {
                    'subject': {
                        'value': triple['subject']['value'], # the subject of the triple
                        'type': 'uri'
                    }, 
                    'predicate': {
                        'value': "a different predicate", # the predicate of the triple
                        'type': 'uri'
                    }, 
                    'object': {
                        'value': triple['object']['value'], # mapped uri
                        'type': 'uri'
                    }
                }
            )
        
        if ptype == 'thesis':
            # nothing needs to happen
            pass
        elif ptype == 'generic':
            for vocab in vocabs["type"]:
                # mint a new triple with the mapped type
                if triple['object']['value'] in vocab["mapping"]:
                    self.output.append(
                        {
                            'subject': {
                                'value': triple['subject']['value'], # the subject of the triple
                                'type': 'uri'
                            }, 
                            'predicate': {
                                'value': triple['predicate']['value'], # the predicate of the triple
                                'type': 'uri'
                            }, 
                            'object': {
                                'value': vocab["uri"], # mapped uri
                                'type': 'uri'
                            }
                        }
                    )
                if "Draft-Submitted" in triple['object']['value']:
                    self.output.append( 
                        {
                            'subject': {
                                'value': triple['subject']['value'], # the subject of the triple
                                'type': 'uri'
                            }, 
                            'predicate': {
                                'value': "http://purl.org/ontology/bibo/status", # the predicate of the triple
                                'type': 'uri'
                            }, 
                            'object': {
                                'value': "http://vivoweb.org/ontology/core#submitted", # mapped uri
                                'type': 'uri'
                            }
                        },
                         {
                            'subject': {
                                'value': triple['subject']['value'], # the subject of the triple
                                'type': 'uri'
                            }, 
                            'predicate': {
                                'value': "http://purl.org/ontology/bibo/status", # the predicate of the triple
                                'type': 'uri'
                            }, 
                            'object': {
                                'value': "http://purl.org/ontology/bibo/status#draft", # mapped uri
                                'type': 'uri'
                            }
                        }
                    )
                if "Published" in triple['object']['value']:
                    self.output.append( 
                         {
                            'subject': {
                                'value': triple['subject']['value'], # the subject of the triple
                                'type': 'uri'
                            }, 
                            'predicate': {
                                'value': "http://purl.org/ontology/bibo/status", # the predicate of the triple
                                'type': 'uri'
                            }, 
                            'object': {
                                'value': "http://purl.org/ontology/bibo/status#published", # mapped uri
                                'type': 'uri'
                            }
                        }
                    )
        elif (ptype == 'community') or (ptype == 'collection'):
            self.output.append(triple)
        
        return self.output

##  QUERY BUILDER
##### Pulls current mappings from triplestore, dynamically builds queries in managable sizes

In [687]:
class Query(object):
    """ Query objects are dynamically generated, and contain SPARQL CONSTRUCT queries with input from the jupiter application profile """
    def __init__(self, ptype, sparqlTerms=sparqlTerms, sparqlData=sparql_mig_test):
        self.mapping = []
        self.sparqlTerms = sparqlTerms  # doesn't need to change (the terms store doesn't change)
        self.sparqlData = sparqlData  # sets the triple store from which to get data (simple, test, or dev)
        self.queries = []
        self.prefixes = ""
        self.filename = ""
        for ns in mig_ns:
            self.prefixes = self.prefixes + " PREFIX %s: <%s> " % (ns['prefix'], ns['uri'])
        self.getMappings()
        print(self.ptype)
        self.generateQueries()
        print(self.queries[0][0])

    def getMappings(self):
        query = "prefix ual: <http://terms.library.ualberta.ca/>SELECT * WHERE {GRAPH ual:%s {?newProperty ual:backwardCompatibleWith ?oldProperty} }" % (self.ptype)
        self.sparqlTerms.setReturnFormat(JSON)
        self.sparqlTerms.setQuery(query)
        results = self.sparqlTerms.query().convert()
        for result in results['results']['bindings']:
            self.mapping.append((result['newProperty']['value'], result['oldProperty']['value']))

    def generateQueries(self):
        pass
    
    def splitQuery(self, result):
        return "/".join(result['resource']['value'].split('/')[:7])


class Collection(Query):
    def __init__(self):
        self.ptype = 'collection'
        self.construct = "CONSTRUCT { ?resource info:hasModel 'IRItem'^^xsd:string ; rdf:type pcdm:Collection"
        self.where = ["WHERE { ?resource info:hasModel 'Collection'^^xsd:string . OPTIONAL { ?resource ualids:is_community 'false'^^xsd:boolean } . OPTIONAL { ?resource ualid:is_community 'false'^^xsd:boolean } . OPTIONAL { ?resource ual:is_community 'false'^^xsd:boolean }"]
        self.select = None
        super().__init__(self.ptype)

    def generateQueries(self):
        for where in self.where:
            construct = self.construct
            for pair in self.mapping:
                construct = "%s ; <%s> ?%s" % (construct, pair[0], removeNS(pair[0]))
                where = " %s . OPTIONAL { ?resource <%s> ?%s . FILTER (?%s!='') }" % (where, pair[1], removeNS(pair[0]), removeNS(pair[0]))
            self.queries.append( ("%s %s } %s }" % (self.prefixes, construct, where), "" ) )


class Community(Query):
    def __init__(self):
        self.ptype = 'community'
        self.construct = "CONSTRUCT { ?resource info:hasModel 'IRItem'^^xsd:string ; rdf:type pcdm:Object; rdf:type ual:Community"
        self.where = ["WHERE { ?resource info:hasModel 'Collection'^^xsd:string ; OPTIONAL { ?resource ualids:is_community 'true'^^xsd:boolean } . OPTIONAL { ?resource ualid:is_community 'true'^^xsd:boolean } . OPTIONAL { ?resource ual:is_community 'true'^^xsd:boolean }"]
        self.select = None
        super().__init__(self.ptype)

    def generateQueries(self):
        for where in self.where:
            construct = self.construct
            for pair in self.mapping:
                construct = "%s ; <%s> ?%s" % (construct, pair[0], removeNS(pair[0]))
                where = " %s . OPTIONAL { ?resource <%s> ?%s . FILTER (?%s!='') }" % (where, pair[1], removeNS(pair[0]), removeNS(pair[0]))
            self.queries.append( ( "%s %s } %s }" % (self.prefixes, construct, where), "" ) )


class Generic(Query):
    def __init__(self):
        self.ptype = 'generic'
        self.construct = "CONSTRUCT { ?resource info:hasModel 'IRItem'^^xsd:string ; rdf:type pcdm:Object; rdf:type works:work"
        self.where = []
        self.select = "SELECT distinct ?collection WHERE { ?resource info:hasModel 'GenericFile'^^xsd:string ; dcterm:type ?type . filter(?type != 'Thesis'^^xsd:string) . ?resource ualids:hasCollectionId ?collection }"
        super().__init__(self.ptype)

    def generateQueries(self):
        query = "%s %s" % (self.prefixes, self.select)
        self.sparqlData.setReturnFormat(JSON)
        self.sparqlData.setQuery(query)
        results = self.sparqlData.query().convert()
        for result in results['results']['bindings']:
            where = "WHERE {  ?resource ualids:hasCollectionId '%s'^^xsd:string ; info:hasModel 'GenericFile'^^xsd:string ; dcterm:type ?type . filter(?type != 'Thesis'^^xsd:string)" % (result['collection']['value'])
            construct = self.construct
            for pair in self.mapping:
                construct = "%s ; <%s> ?%s" % (construct, pair[0], removeNS(pair[0]))
                where = " %s . OPTIONAL { ?resource <%s> ?%s . FILTER (?%s!='') }" % (where, pair[1], removeNS(pair[0]), removeNS(pair[0]))
            self.queries.append( ( "%s %s } %s }" % (self.prefixes, construct, where), result['collection']['value'] ) )


class Thesis(Query):
    def __init__(self):
        self.ptype = 'thesis'
        self.construct = "CONSTRUCT { ?resource info:hasModel 'IRItem'^^xsd:string ; rdf:type pcdm:Object; rdf:type works:work ; rdf:type bibo:Thesis"
        self.where = []
        self.select = "SELECT distinct ?year_created WHERE { ?resource info:hasModel 'GenericFile'^^xsd:string ; dcterm:type 'Thesis'^^xsd:string ; ualids:year_created ?year_created }"
        super().__init__(self.ptype)

    def generateQueries(self):
        query = "%s %s" % (self.prefixes, self.select)
        self.sparqlData.setReturnFormat(JSON)
        self.sparqlData.setQuery(query)
        results = self.sparqlData.query().convert()
        for result in results['results']['bindings']:
            where = "WHERE { ?resource ualids:year_created '%s'^^xsd:string ; info:hasModel 'GenericFile'^^xsd:string ; dcterm:type 'Thesis'^^xsd:string" % (result['year_created']['value'])
            construct = self.construct
            for pair in self.mapping:
                construct = "%s ; <%s> ?%s" % (construct, pair[0], removeNS(pair[0]))
                where = " %s . OPTIONAL { ?resource <%s> ?%s . FILTER (?%s!='') }" % (where, pair[1], removeNS(pair[0]), removeNS(pair[0]))
            self.queries.append( ( "%s %s } %s }" % (self.prefixes, construct, where), result['year_created']['value'] ))


class Batch(Query):
    def __init__(self):
        self.ptype = 'batch'
        self.construct = "CONSTRUCT { ?resource info:hasModel 'Batch' ; schema:result ?type; ?predicate ?object }"
        self.where = "WHERE { ?resource info:hasModel 'Batch'^^xsd:string ; "
        self.select = "SELECT distinct ?resource WHERE  { ?resource info:hasModel 'Batch'^^xsd:string } "
        super().__init__(self.ptype)
    
    def generateQueries(self):
        query = "%s %s" % (self.prefixes, self.select)
        self.sparqlData.setReturnFormat(JSON)
        self.sparqlData.setQuery(query)
        results = self.sparqlData.query().convert()
        for result in results['results']['bindings']:
            group = self.splitQuery(result)
            self.queries.append( ( "%s %s %s FILTER (contains(str(?resource), '%s')) . ?resource dcterm:type ?type . ?resource ?predicate ?object .  . FILTER (?object!='') }" % (self.prefixes, self.construct, self.where, group), result['resource']['value'].split('/')[6] ) )

##  DATA TRANSPORT OBJECTS
##### Runs a query, sends data to get transformed, saves data to appropriate file

In [688]:
class Data(object):
    def __init__(self, q, sparqlData, sparqlTerms, ptype):
        self.q = q
        self.query = q[0]
        self.sparqlData = sparqlData
        self.sparqlTerms = sparqlTerms
        self.filename = ""
        self.output = []
        self.ptype = ptype

    def transformData(self):
        self.sparqlData.setMethod("GET")
        self.sparqlData.setReturnFormat(JSON)
        self.sparqlData.setQuery(self.query)
        results = self.sparqlData.query().convert()['results']['bindings']
        for result in results:
            try:
                result = TransformationFactory().getTransformation(result, self.ptype)
            except:
                PrintException()
            try:
                if isinstance(result, list):
                    for triple in result:
                        s = "<%s>" % (str(triple['subject']['value']))
                        p = "<%s>" % (str(triple['predicate']['value']))
                        if triple['object']['type'] == 'uri':
                            o = "<%s>" % (str(triple['object']['value']))
                        else:
                            o = "\"%s\"" % (str(triple['object']['value']))
                        self.output.append("%s %s %s . \n" % (s, p, o))
            except:
                PrintException()
        try:
            with open(self.filename, "w+") as f:
                f.writelines(self.output)
        except:
            PrintException()

        #with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        #    future_to_result = {executor.submit(self.processResults, results, query): result for result in results}
        #    for future in concurrent.futures.as_completed(future_to_result):
        #        result = future_to_result[future]
        #        try:
        #            future.result()
        #        except Exception:
        #            PrintException()

class CollectionData(Data):
    def __init__(self, q, sparqlData, sparqlTerms, ptype):
        super().__init__(q, sparqlData, sparqlTerms, ptype)
        self.filename = 'results/collection.nt'

class CommunityData(Data):
    def __init__(self, q, sparqlData, sparqlTerms, ptype):
        super().__init__(q, sparqlData, sparqlTerms, ptype)
        self.filename = 'results/community.nt'


class ThesisData(Data):
    def __init__(self, q, sparqlData, sparqlTerms, ptype):
        super().__init__(q, sparqlData, sparqlTerms, ptype)
        self.filename = "results/thesis/%s.nt" % (self.q[1])


class GenericData(Data):
    def __init__(self, q, sparqlData, sparqlTerms, ptype):
        super().__init__(q, sparqlData, sparqlTerms, ptype)
        self.filename = "results/generic/%s.nt" % (self.q[1])

class BatchData(Data):
    def __init__(self, q, sparqlData, sparqlTerms, ptype):
        super().__init__(q, sparqlData, sparqlTerms, ptype)
        self.filename = "results/batch/%s.nt" % (self.q[1])

In [689]:
class QueryFactory():
    @staticmethod
    def getMigrationQuery(ptype):
        """ returns a specified query object depending on the type passed in"""
        functions = {"collection": Collection(), 
                     "community": Community(), 
                     "thesis": Thesis(), 
                     "generic": Generic(), 
                     "batch": Batch()
                    }
        if ptype in functions:
            return functions[ptype]
        else:
            return None

In [690]:
class DataFactory():
    @staticmethod
    def getData(q, mig):
        """ returns a specified query object depending on the type passed in"""
        functions = {"collection": CollectionData(q, mig.sparqlData, mig.sparqlTerms, mig.ptype), 
                     "community": CommunityData(q, mig.sparqlData, mig.sparqlTerms, mig.ptype), 
                     "thesis": ThesisData(q, mig.sparqlData, mig.sparqlTerms, mig.ptype), 
                     "generic": GenericData(q, mig.sparqlData, mig.sparqlTerms, mig.ptype), 
                     "batch": BatchData(q, mig.sparqlData, mig.sparqlTerms, mig.ptype)
                    }
        if mig.ptype in functions:
            return functions[mig.ptype]
        else:
            return None

In [691]:
class TransformationFactory():
    @staticmethod
    def getTransformation(triple, ptype):
        function = re.sub(r'[0-9]+', '', triple['predicate']['value'].split('/')[-1].replace('#', '').replace('-', ''))
        functions = {"rdfsyntaxnstype": Transformation().rdfsyntaxnstype(triple, ptype),
                     "language": Transformation().language(triple, ptype),
                     "type": Transformation().type(triple, ptype),
                     "rights": Transformation().rights(triple, ptype),
                     "license": Transformation().license(triple, ptype)
                    }
        if function in functions:
            return functions[function]
        else:
            return [triple]

In [692]:
if __name__ == "__main__":
	main()

collection
 PREFIX premis: <http://www.loc.gov/premis/rdf/v1#>  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  PREFIX ual: <http://terms.library.ualberta.ca/>  PREFIX ualids: <http://terms.library.ualberta.ca/identifiers/>  PREFIX ualid: <http://terms.library.ualberta.ca/id/>  PREFIX ualdate: <http://terms.library.ualberta.ca/date/>  PREFIX ualrole: <http://terms.library.ualberta.ca/role/>  PREFIX ualthesis: <http://terms.library.ualberta.ca/thesis/>  PREFIX info: <info:fedora/fedora-system:def/model#>  PREFIX dcterm: <http://purl.org/dc/terms/>  PREFIX pcdm: <http://pcdm.org/models#>  PREFIX works: <http://projecthydra.org/works/models#>  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  PREFIX fedora: <http://fedora.info/definitions/v4/repository#>  PREFIX iana: <http://www.iana.org/assignments/relation/>  PREFIX dc: <http://purl.org/dc/elements/1.1/>  PREFIX acl: <http://projecthydra.org/ns/auth/acl#>  PREFIX webacl: <http://www.w3.org/ns/auth/acl#>  PREFIX scholar: 

 PREFIX premis: <http://www.loc.gov/premis/rdf/v1#>  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  PREFIX ual: <http://terms.library.ualberta.ca/>  PREFIX ualids: <http://terms.library.ualberta.ca/identifiers/>  PREFIX ualid: <http://terms.library.ualberta.ca/id/>  PREFIX ualdate: <http://terms.library.ualberta.ca/date/>  PREFIX ualrole: <http://terms.library.ualberta.ca/role/>  PREFIX ualthesis: <http://terms.library.ualberta.ca/thesis/>  PREFIX info: <info:fedora/fedora-system:def/model#>  PREFIX dcterm: <http://purl.org/dc/terms/>  PREFIX pcdm: <http://pcdm.org/models#>  PREFIX works: <http://projecthydra.org/works/models#>  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  PREFIX fedora: <http://fedora.info/definitions/v4/repository#>  PREFIX iana: <http://www.iana.org/assignments/relation/>  PREFIX dc: <http://purl.org/dc/elements/1.1/>  PREFIX acl: <http://projecthydra.org/ns/auth/acl#>  PREFIX webacl: <http://www.w3.org/ns/auth/acl#>  PREFIX scholar: <http://sch

batch
 PREFIX premis: <http://www.loc.gov/premis/rdf/v1#>  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  PREFIX ual: <http://terms.library.ualberta.ca/>  PREFIX ualids: <http://terms.library.ualberta.ca/identifiers/>  PREFIX ualid: <http://terms.library.ualberta.ca/id/>  PREFIX ualdate: <http://terms.library.ualberta.ca/date/>  PREFIX ualrole: <http://terms.library.ualberta.ca/role/>  PREFIX ualthesis: <http://terms.library.ualberta.ca/thesis/>  PREFIX info: <info:fedora/fedora-system:def/model#>  PREFIX dcterm: <http://purl.org/dc/terms/>  PREFIX pcdm: <http://pcdm.org/models#>  PREFIX works: <http://projecthydra.org/works/models#>  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  PREFIX fedora: <http://fedora.info/definitions/v4/repository#>  PREFIX iana: <http://www.iana.org/assignments/relation/>  PREFIX dc: <http://purl.org/dc/elements/1.1/>  PREFIX acl: <http://projecthydra.org/ns/auth/acl#>  PREFIX webacl: <http://www.w3.org/ns/auth/acl#>  PREFIX scholar: <http

 PREFIX premis: <http://www.loc.gov/premis/rdf/v1#>  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  PREFIX ual: <http://terms.library.ualberta.ca/>  PREFIX ualids: <http://terms.library.ualberta.ca/identifiers/>  PREFIX ualid: <http://terms.library.ualberta.ca/id/>  PREFIX ualdate: <http://terms.library.ualberta.ca/date/>  PREFIX ualrole: <http://terms.library.ualberta.ca/role/>  PREFIX ualthesis: <http://terms.library.ualberta.ca/thesis/>  PREFIX info: <info:fedora/fedora-system:def/model#>  PREFIX dcterm: <http://purl.org/dc/terms/>  PREFIX pcdm: <http://pcdm.org/models#>  PREFIX works: <http://projecthydra.org/works/models#>  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  PREFIX fedora: <http://fedora.info/definitions/v4/repository#>  PREFIX iana: <http://www.iana.org/assignments/relation/>  PREFIX dc: <http://purl.org/dc/elements/1.1/>  PREFIX acl: <http://projecthydra.org/ns/auth/acl#>  PREFIX webacl: <http://www.w3.org/ns/auth/acl#>  PREFIX scholar: <http://sch

 PREFIX premis: <http://www.loc.gov/premis/rdf/v1#>  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  PREFIX ual: <http://terms.library.ualberta.ca/>  PREFIX ualids: <http://terms.library.ualberta.ca/identifiers/>  PREFIX ualid: <http://terms.library.ualberta.ca/id/>  PREFIX ualdate: <http://terms.library.ualberta.ca/date/>  PREFIX ualrole: <http://terms.library.ualberta.ca/role/>  PREFIX ualthesis: <http://terms.library.ualberta.ca/thesis/>  PREFIX info: <info:fedora/fedora-system:def/model#>  PREFIX dcterm: <http://purl.org/dc/terms/>  PREFIX pcdm: <http://pcdm.org/models#>  PREFIX works: <http://projecthydra.org/works/models#>  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  PREFIX fedora: <http://fedora.info/definitions/v4/repository#>  PREFIX iana: <http://www.iana.org/assignments/relation/>  PREFIX dc: <http://purl.org/dc/elements/1.1/>  PREFIX acl: <http://projecthydra.org/ns/auth/acl#>  PREFIX webacl: <http://www.w3.org/ns/auth/acl#>  PREFIX scholar: <http://sch

 PREFIX premis: <http://www.loc.gov/premis/rdf/v1#>  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  PREFIX ual: <http://terms.library.ualberta.ca/>  PREFIX ualids: <http://terms.library.ualberta.ca/identifiers/>  PREFIX ualid: <http://terms.library.ualberta.ca/id/>  PREFIX ualdate: <http://terms.library.ualberta.ca/date/>  PREFIX ualrole: <http://terms.library.ualberta.ca/role/>  PREFIX ualthesis: <http://terms.library.ualberta.ca/thesis/>  PREFIX info: <info:fedora/fedora-system:def/model#>  PREFIX dcterm: <http://purl.org/dc/terms/>  PREFIX pcdm: <http://pcdm.org/models#>  PREFIX works: <http://projecthydra.org/works/models#>  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  PREFIX fedora: <http://fedora.info/definitions/v4/repository#>  PREFIX iana: <http://www.iana.org/assignments/relation/>  PREFIX dc: <http://purl.org/dc/elements/1.1/>  PREFIX acl: <http://projecthydra.org/ns/auth/acl#>  PREFIX webacl: <http://www.w3.org/ns/auth/acl#>  PREFIX scholar: <http://sch